In [1]:
## load the packages here
import numpy as np
import pandas as pd 
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn import metrics
import statsmodels.api as sm 
import statsmodels.formula.api as smf


## Task 1: Download your data and load them into the Python.
You can find them [**here**](https://drive.google.com/file/d/0Bz9_0VdXvv9bX0MzUEhVdmpCc3c/view?usp=sharing).

- Features and response variables are in different files
- be careful about number of spaces between the values in the file

In [2]:
# get the features from the file features.txt
features = list()
with open('features.txt') as f:
    features = [line for line in f.readlines()]
print('No of Features: {}'.format(len(features)))

No of Features: 561


In [3]:
X=pd.read_csv('X_train.txt', delim_whitespace=True, header=None, names=features)

In [4]:
y=pd.read_csv('y_train.txt', delim_whitespace=True, header=None)

## Task 2: Create Binary target variable: categories 1,2,3 --> 1, categories 4,5,6 --> 0 
This will represent binary variable indicating if person is walking or not

In [5]:
y = y.replace([4,5,6],0)

In [6]:
y = y.replace([1,2,3],1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)


## Task 3

+ Create Univariate Binary Logistic Regression with feature number 54, which represents `tGravityAcc-min()-Y`: gravity acceleration signals in direction of Y
+ Compare the results of Logistic regressions from different Python packages (sklearn, statsmodel)
+ Plot the FIT of predicted probabilities to the original values

In [8]:
X_train.iloc[:, 53]

2591    0.056201
6002   -0.133582
5381   -0.126702
4338   -0.338458
6597   -0.214790
          ...   
4931    0.083024
3264   -0.168234
1653    0.247515
2607    0.902883
2732    0.350170
Name: 54 tGravityAcc-min()-Y\n, Length: 5881, dtype: float64

In [9]:
# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression()
X_train_53 = X_train.iloc[:, 53]
X_test_53 = X_test.iloc[:, 53]

In [10]:
# fit the model with data
logreg.fit(X_train_53.values.reshape(-1,1),y_train)

#
y_pred_53=logreg.predict(X_test_53.values.reshape(-1,1))

C:\Users\jesse\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [11]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred_53)
cnf_matrix

array([[595, 243],
       [101, 532]], dtype=int64)

In [20]:
metrics.f1_score(y_test, y_pred_53)

0.7556818181818181

In [39]:
# importing libraries
import statsmodels.api as sm
import pandas as pd 

log_reg = sm.Logit(y_train,X_train_53).fit()


Optimization terminated successfully.
         Current function value: 0.492367
         Iterations 7


In [16]:
y_pred_53_stat = log_reg.predict(X_test_53)
prediction = list(map(round, y_pred_53_stat))
cnf_matrix = metrics.confusion_matrix(y_test, prediction)
cnf_matrix


array([[498, 340],
       [ 13, 620]], dtype=int64)

In [21]:
metrics.f1_score(y_test, prediction)

0.7784055241682362

## Task 4
- Try to fit Binary Logistic Regression with all the features? How many are significant?

In [29]:
data = X_train.copy()

In [30]:
corr = data.corr()

In [31]:
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False
selected_columns = data.columns[columns]
data = data[selected_columns]

In [36]:
data.shape

(5881, 219)

In [43]:
selected_columns = selected_columns[1:]

def backwardElimination(x, Y, sl, columns):
    numVars = len(x[0])
    for i in range(0, numVars):
        regressor_OLS = LogisticRegression(Y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    x = np.delete(x, j, 1)
                    columns = np.delete(columns, j)
                    
    regressor_OLS.summary()
    return x, columns
SL = 0.05
data_modeled, selected_columns = backwardElimination(data.values, y_train.values, SL, selected_columns)

         Current function value: inf
         Iterations: 35


LinAlgError: Singular matrix

In [41]:
result = pd.DataFrame()
result['diagnosis'] = data.iloc[:,0]

In [42]:
data = pd.DataFrame(data = data_modeled, columns = selected_columns)

NameError: name 'data_modeled' is not defined

In [60]:
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression()
logreg.fit(X_train,y_train)

C:\Users\jesse\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\jesse\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [61]:
y_predict = logreg.predict(X_test)

In [81]:
metrics.f1_score(y_test, y_predict,average=None)

array([0.99383984, 0.99484536, 0.99744246, 0.96564195, 0.96564195,
       1.        ])

In [82]:
(logreg.coef_).shape

(6, 561)

In [75]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           1       1.00      0.99      0.99       244
           2       0.99      1.00      0.99       193
           3       1.00      0.99      1.00       196
           4       0.96      0.97      0.97       275
           5       0.97      0.96      0.97       278
           6       1.00      1.00      1.00       285

    accuracy                           0.99      1471
   macro avg       0.99      0.99      0.99      1471
weighted avg       0.99      0.99      0.99      1471



In [76]:
from sklearn.feature_selection import SelectFromModel

model = SelectFromModel(logreg, prefit=True)
X_new = model.transform(X_train)

In [80]:
logreg.fit(X_new,y_train)
y_predict_1 = logreg.predict(X_)

ValueError: X has 189 features per sample; expecting 561

## Task 5
Now, let's fit Multinomial Logistic regression to predict all categories. Firstly, we can start with **Univariate** model for these features number separately:
+ 4
+ 54
- 19

Check the contingency matrix to see the effect of particular features!! (each feature can be good in predicting different categories)

0    6
dtype: int64

## Task 6

Fit the Multinomial Logistic Regression model again. Now, try to choose **all** important features we have in the dataset. Who will get the best predictions with the smallest number of features?

## Task 7 (Stretch)
Create your own function for Stepwise selection. Use either sklearn or statsmodel.